In [3]:
import pandas as pd
import spacy
#pip install nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re

text = u'This is a smiley face \U0001f602'
print(text) # with emoji

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

print(deEmojify(text))

In [2]:
import re

text = u'This is a smiley face \U0001f602'
print(text) # with emoji

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

print(deEmojify(text))

This is a smiley face 😂
This is a smiley face 


# Pre-process the text

In [3]:
def preprocess(corpus):
    '''This pre-processing function lemmatizes words, makes a commment case-insensitive, removes punctuation,
    removes stop words and removes single characters. '''
    
    # convert df to list; *corpus* is of a form of a list of str
    original_indices = corpus.index
    corpus = corpus.to_list()
    
    # initialize lemmatizer and a list of stopwords
    lemmatizer = spacy.load('en_core_web_sm', disable = ['parser', 'ner'])
    
    for i in range(len(corpus)):
        text = corpus[i] # one comment
        
        # convert text to lower case
        text = text.lower()
        
        # remove emoji
        text = deEmojify(text)
        
        # remove punctuations and numbers but save the "'"
        symbols = "—…,!\"#$%&()*+-./:;<=>?@[\]^_`{|}~0123456789"
        for sym in symbols:
            text = np.char.replace(text, sym, ' ') # replace symbols by one space

        # tokenize text
        text_tokens = word_tokenize(str(text))

        # spaCy Lemmatization
        doc = lemmatizer(" ".join([tk for tk in text_tokens]))
        #print(doc)
        corpus[i] = " ".join([token.lemma_ for token in doc]) # Extract the lemma for each token and join
    return pd.Series(corpus, index=original_indices)

In [3]:
def preprocess_rmsw(corpus):
    '''remove stopword'''
    '''This pre-processing function lemmatizes words, makes a commment case-insensitive, removes punctuation,
    removes stop words and removes single characters. '''
    
    # convert df to list; *corpus* is of a form of a list of str
    corpus = corpus.to_list()
    
    # initialize lemmatizer and a list of stopwords
    lemmatizer = spacy.load('en_core_web_sm', disable = ['parser', 'ner'])
    all_stopwords = lemmatizer.Defaults.stop_words
    print(all_stopwords)
    
    for i in range(len(corpus)):
        text = corpus[i] # one comment
        
        # convert text to lower case
        text = text.lower()
        
        # remove punctuations and numbers but save the "'"
        symbols = "—…,!\"#$%&()*+-./:;<=>?@[\]^_`{|}~0123456789"
        for sym in symbols:
            text = np.char.replace(text, sym, ' ') # replace symbols by one space

        # tokenize text
        text_tokens = word_tokenize(str(text))
        
        # remove stop words ("sw") and single characters
        tokens_without_sw = " ".join([word for word in text_tokens if (word not in all_stopwords) and (len(word) > 1)])

        # remove "’"
        tokens_without_sw = np.char.replace(tokens_without_sw, "'", ' ')
        
        # spaCy Lemmatization
        doc = lemmatizer(str(tokens_without_sw))
        
        corpus[i] = " ".join([token.lemma_ for token in doc]) # Extract the lemma for each token and join
    return corpus